# Setup

In [ ]:
# Parameters
keyframes_dir = None
save_dir = None
bs = None

In [ ]:
import os

dir_path = os.getcwd()

if not keyframes_dir:
    if 'google.colab' in str(get_ipython()):
        # Update this path as necessary
        keyframes_dir = f'{dir_path}/keyframes'
    elif 'kaggle' in str(get_ipython()):
        keyframes_dir = f'{dir_path}/keyframes'
    else:
        parent_dir_path = os.path.dirname(dir_path)
        keyframes_dir = f'{parent_dir_path}/transnet/keyframes'

if not bs:
    bs = 4

if not save_dir:
    save_dir = './CLIPv2_features'

In [ ]:
# Instal dependency
! pip install open_clip_torch

In [ ]:
# Import module
import os
import open_clip
import glob
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm
import json

/home/heigatvu/my-project/competition/HCMC-AI/pipeline-hcm-ai/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Parse data path

In [ ]:
def parse_keyframe_info(keyframes_dir):
    all_keyframe_paths = {}
    for part in sorted(os.listdir(keyframes_dir)):
        data_part_path = f'{keyframes_dir}/{part}'
        data_part = part.split('/')[-1]
        all_keyframe_paths[data_part] = []
        image_path = sorted(glob.glob(f'{data_part_path}/*.jpg'))
        all_keyframe_paths[data_part] = image_path
    return all_keyframe_paths

# Model

In [ ]:
##### Load Model #####
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model, _, preprocess = open_clip.create_model_and_transforms('ViT-L-14', device=device, pretrained='datacomp_xl_s13b_b90k')

cuda


In [ ]:
def create_directory(path):
    """Create a directory if it does not exist."""
    if not os.path.exists(path):
        os.makedirs(path)


def process_and_save_results(all_keyframe_paths, save_dir, model, bs):
    create_directory(save_dir)
    
    for video_id, video_keyframe_paths in tqdm(all_keyframe_paths.items()):
        video_feats = []
        # for video_id in tqdm(video_ids):
        #     video_feats = []
        #     video_keyframe_path = video_keyframe_paths
        for i in range(0, len(video_keyframe_paths), bs):
            # Support batchsize inferencing
            images = []
            image_paths = video_keyframe_paths[i:i+bs]
            for image_path in image_paths:
                image = preprocess(Image.open(image_path)).unsqueeze(0)
                images.append(image)
            images = torch.cat(images).to(device)

            with torch.no_grad(), torch.cuda.amp.autocast():
                image_feats = model.encode_image(images)
            image_feats /= image_feats.norm(dim=-1, keepdim=True)

            for b in range(image_feats.shape[0]):
                video_feats.append(image_feats[b].detach().cpu().numpy().astype(np.float32).flatten())
        
        np.save(f'{save_dir}/{video_id}.npy', video_feats)

In [ ]:
all_keyframe_paths = parse_keyframe_info(keyframes_dir)
process_and_save_results(all_keyframe_paths, save_dir, model, bs)

100%|██████████| 4/4 [01:25<00:00, 21.47s/it]
